In [46]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from config import *

In [13]:
df = pd.read_excel(filename_all)

df['Date']  = pd.to_datetime(df['Date'], format = '%d/%m/%Y')
df['Week']  = df['Date'].dt.isocalendar().week

df['week_cos'] = np.cos(2 * np.pi * df['Week'] / 7)
df['week_sin'] = np.sin(2 * np.pi * df['Week'] / 7)

In [97]:
df = df[['week_cos', 'week_sin', 'first']]

In [98]:
cols_to_norm = ['week_cos', 'week_sin']

In [99]:
df[cols_to_norm] = df[cols_to_norm].apply(lambda x : (x - x.min()) / (x.max() - x.min()))

In [100]:
week_cos = tf.feature_column.numeric_column('week_cos')
week_sin = tf.feature_column.numeric_column('week_sin')

In [18]:
day_name = tf.feature_column.categorical_column_with_vocabulary_list('Day_Name', ['Tuesday', 'Wednesday', 'Thursday', 'Saturday'])

In [101]:
feat_cols = [week_cos, week_sin]

In [102]:
x_data = df.drop('first', axis=1)

In [103]:
labels = df['first']

In [104]:
X_train, X_test, y_train, y_test = train_test_split(x_data, labels, test_size = test_size, random_state = rand_state)

In [105]:
embedded_day = tf.feature_column.embedding_column(day_name, dimension = 4)

In [106]:
feat_cols = [week_cos, week_sin]

In [107]:
input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size = 10, num_epochs = 1000, shuffle = True)

In [108]:
dnn_model = tf.estimator.DNNClassifier(hidden_units = [20,20,20], feature_columns = feat_cols, n_classes = 42)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\jcaraan\\AppData\\Local\\Temp\\tmppr345d2x', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [109]:
dnn_model.train(input_fn = input_func, steps = 1000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\jcaraan\AppData\Local\Temp\tmppr345d2x\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 3.7844589, step = 0
INFO:tensorflow:global_step/sec: 354.582
INFO:tensorflow:loss = 3.7332668, step = 100 (0.284 sec)
INFO:tensorflow:global_step/sec: 483.059
INFO:tensorflow:loss = 3.7586093, step = 200 (0.207 sec)
INFO:tensorflow:global_step/sec: 512.776
INFO:tensorflow:loss = 3.7602706, step = 300 (0.196 sec)
INFO:tensorflow:global_step/sec: 485.406
INFO:tensorflow:loss = 3.6763954, step = 400 (0.205 sec)
INFO:tensorflow:global_step/sec: 469.445
INFO:tensorflow:loss = 3.685094,

In [112]:
eval_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn( x = X_test, y = y_test, batch_size = 10, num_epochs = 100, shuffle = False)

In [113]:
dnn_model.evaluate(eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-03-15T17:20:49Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\jcaraan\AppData\Local\Temp\tmppr345d2x\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 6.09246s
INFO:tensorflow:Finished evaluation at 2021-03-15-17:20:55
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.07716049, average_loss = 3.6494093, global_step = 1000, loss = 3.6494095
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\jcaraan\AppData\Local\Temp\tmppr345d2x\model.ckpt-1000


{'accuracy': 0.07716049,
 'average_loss': 3.6494093,
 'loss': 3.6494095,
 'global_step': 1000}